In [1]:
import os
import io
import glob
import json
import re
import numpy as np
import pandas as pd

os.chdir(os.path.expanduser("~/data/omop"))

In [2]:
# Load all tables into dataframes
tables = {re.findall(r"tsv\/(.*?)_OMOP.*?", f)[0]: 
          pd.read_table(f, index_col=0, dtype={"MRN": str}).round(5)
          for f in glob.glob("tsv/*.tsv")}

# Drop sig column as it is free text notes on administration of drug
tables["drug_exposure"] = tables["drug_exposure"].drop(columns="sig")

In [3]:
# Replace numerical NaN with 0 and text with None so R and Javascript can import without errors
tables["drug_exposure"] = tables["drug_exposure"].fillna({
    'drug_start_in_days': 0,
    'drug_end_in_days': 0,
    'drug_concept_id': 0,
    'drug_type_concept_id': 0,
    'days_supply': 0,
#     'dose_unit_source_value': None,
    'quantity': 0,
    'refills': 0
}).replace(dict(dose_unit_source_value={np.nan: None}))

In [4]:
# List all the tables we found
tables.keys()

dict_keys(['drug_exposure', 'death', 'person', 'condition_occurrence', 'procedure_occurrence'])

In [5]:
# Organize by patient mrn
patients = {mrn: {name: table.loc[mrn].to_dict() if name == "person" or name == "death"
                  else table.loc[mrn].to_dict("records")
                  for name, table in tables.items() if mrn in table.index}
            for mrn in tables["person"].index}

In [6]:
mrn_to_cgt_id = pd.read_table(io.StringIO("""
DD3EBE5F8FE593	f9b6a782-bbf5-4be8-bf7e-d1a9586d9552
D2DF3E8C7EE52D	c2e2e081-4c39-4201-8a27-7b469ed39490
D7A1DB622B2D86	db2d85aa-4f94-4e77-8755-6b94a710c1aa
D8C5D297AA5EE3	2fbc25da-3965-49c4-866f-72cf0abc2417
D92E7044C471BD	940171e7-d358-463a-8d9a-2b2fa90c2a84
D27DA93D449FD3	f0314175-2d19-4146-8754-fc5aed3ab420
D9A57FDBDDEE4F	c7dbcfac-37ea-43f8-8899-1a9f2fb56341
D3622CCFB68372	ef5c3164-6f45-4d3a-88f0-4509226c5571
D223AAE0137F8D	ec3d977b-c310-4df3-a444-f79bc3dd8b58
D37A33A3E2C50D	131cf62d-ad78-49c1-a699-5bcc1004cd12
D1FB80DF9BD001	cf11c31c-f4c3-48ba-9c46-66f406d0b7a1
D2EFA05E54A47A	ccc2ba97-912f-4b62-b767-cca129ee6a56
D3689FA5A3F1D2	104ec531-5d95-41e2-ac72-f6cff2006b8e
D3F26D1C0E0596	a5627ac3-450d-4036-ade8-99ae62a5c232
DB53107CEFF7EF	5189efbe-3382-4353-ad2f-9afd0255c2c8
D2824A4944E8E4	d199cfb0-91e8-471d-b1b3-53189cd64ee0
"""), names=["mrn", "cgt_id"], index_col=0)

In [7]:
# Only include partipants we plan to upload/export


# patients.keys()
mrn_to_cgt_id

,cgt_id
mrn,
DD3EBE5F8FE593,f9b6a782-bbf5-4be8-bf7e-d1a9586d9552
D2DF3E8C7EE52D,c2e2e081-4c39-4201-8a27-7b469ed39490
D7A1DB622B2D86,db2d85aa-4f94-4e77-8755-6b94a710c1aa
D8C5D297AA5EE3,2fbc25da-3965-49c4-866f-72cf0abc2417
D92E7044C471BD,940171e7-d358-463a-8d9a-2b2fa90c2a84
D27DA93D449FD3,f0314175-2d19-4146-8754-fc5aed3ab420
D9A57FDBDDEE4F,c7dbcfac-37ea-43f8-8899-1a9f2fb56341
D3622CCFB68372,ef5c3164-6f45-4d3a-88f0-4509226c5571
D223AAE0137F8D,ec3d977b-c310-4df3-a444-f79bc3dd8b58


In [12]:
# Dump out omop.json per patient with mrn as the name
os.makedirs("json", exist_ok=True)

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.int64):
            return int(obj)
        else:
            return json.JSONEncoder.default(self, obj)
    
for mrn, omop in patients.items():
    if mrn not in mrn_to_cgt_id.index:
        continue
    with open("json/{}.omop.json".format(mrn_to_cgt_id.loc[mrn].cgt_id), "w") as f:
        f.write(json.dumps(patients[mrn], cls=NumpyEncoder, sort_keys=True))

In [9]:
# Verify round trip
for mrn, _ in patients.items():
    A = json.dumps(patients[mrn], cls=NumpyEncoder, sort_keys=True)
    B = json.dumps(json.loads(A), sort_keys=True)
    assert A == B

## Validate Javascript Reading

Open the debug console in your browser to view the output of the following and verify that you can just eval the generated JSON back into a javascript dictionary. You should see a javascript dictionary in the console that you can expand that has the same values as the python dictionary and source json string.

In [10]:
# # Pass the omop string into the browser via window variable
from IPython.display import Javascript
Javascript("window.omop={};".format(json.dumps(patients[list(patients.keys())[0]], cls=NumpyEncoder, sort_keys=True)))

<IPython.core.display.Javascript object>

In [11]:
%%javascript
// console.log(window.omop)

<IPython.core.display.Javascript object>